source: http://www.jakob-aungiers.com/articles/a/LSTM-Neural-Network-for-Time-Series-Prediction

In [1]:
# only works on keras==1.2.2 tf==0.12.1

In [2]:
import os
import time
import warnings
import numpy as np
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

from numpy import newaxis

Using TensorFlow backend.


In [3]:
filename = 'sp500.csv'
seq_len = 50
normalise_window = True

In [156]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy Numpy warnings

def load_data(filename, seq_len, normalise_window,ratio):
    f = open(filename, 'rb').read()
    data = f.decode().split('\n')[:-1]

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    
    np.random.seed(42)
    
    result = np.array(result)
    np.random.shuffle(result)
    
    
    if normalise_window:
        result = normalise_windows(result)

    result = np.array(result)

    row = round(ratio * result.shape[0])
    print("row",row)
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  

    return [x_train, y_train, x_test, y_test]

def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

In [157]:
def build_model():
    model = Sequential()

    model.add(LSTM(
        input_dim= 1,
        output_dim= 50,
        return_sequences=False))
    model.add(Dropout(0.4))

    model.add(Dense(
        output_dim= 1))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop")
    print("> Compilation Time : ", time.time() - start)
    return model

In [176]:
errors = []
for ratio in [0.005,0.01,0.015,0.2,0.4]:
    X_train, y_train, X_test, y_test = load_data(filename, seq_len, True,ratio)

    # num = 50
    # X_train = X_train[:num,]
    # y_train = y_train[:num]
    print('> Data Loaded. Compiling...')

    model = build_model()
    print("one datapoint from X_train: ", X_train[0][1])

    model.fit(
        X_train,
        y_train,
        batch_size=512,
        nb_epoch=10,
        validation_split=0.05)



    y_te_pred = model.predict(X_test)

    error = np.abs(y_test.reshape((-1,1)) - y_te_pred)

    errors.append(sum(error)/X_test.shape[0])

('row', 21.0)
> Data Loaded. Compiling...
('> Compilation Time : ', 0.025809049606323242)
('one datapoint from X_train: ', array([-0.00768479]))
Train on 19 samples, validate on 2 samples
Epoch 1/10
19/19 [==============================] - 3s - loss: 0.0162 - val_loss: 0.0127
Epoch 2/10
19/19 [==============================] - 0s - loss: 0.0068 - val_loss: 0.0124
Epoch 3/10
19/19 [==============================] - 0s - loss: 0.0071 - val_loss: 0.0090
Epoch 4/10
19/19 [==============================] - 0s - loss: 0.0074 - val_loss: 0.0084
Epoch 5/10
19/19 [==============================] - 0s - loss: 0.0042 - val_loss: 0.0065
Epoch 6/10
19/19 [==============================] - 0s - loss: 0.0026 - val_loss: 0.0058
Epoch 7/10
19/19 [==============================] - 0s - loss: 0.0035 - val_loss: 0.0050
Epoch 8/10
19/19 [==============================] - 0s - loss: 0.0030 - val_loss: 0.0015
Epoch 9/10
19/19 [==============================] - 0s - loss: 0.0038 - val_loss: 0.0047
Epoch 10/10

In [177]:
print errors

[array([ 0.02972256]), array([ 0.01486013]), array([ 0.01827429]), array([ 0.01594325]), array([ 0.01599582])]


import matplotlib.pyplot as plt
plt.plot(range(len(y_test)),y_te_pred)
plt.plot(range(len(y_test)),y_test)
plt.show()

In [114]:
plt.scatter(y_test,error_per)
plt.show()

In [87]:
np.sqrt(np.mean(error**2))/0.56

0.055818890911989341